In [ ]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json

# get key from keypairs.json
# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fouront-mastertest', 'fourfront-hotseat']
# get admin key from s3

# modify the following accordingly
my_key = ff_utils.get_authentication_with_server({}, ff_env = 'data')
transfer_key = get_key('local')
find_linked = ['11f207be-ebc4-4622-8b42-02e7841d17db']

print('WORKING ON', my_key['server'], '\n')
store, uuids = ff_utils.expand_es_metadata(find_linked, key = my_key, add_pc_wfr = True, ignore_field=['references', 'attachments'])
for key in store:
    print(key, len(store[key]))
print(len([i['uuid'] for key in store for i in store[key]]))

# replace workflows from inserts
# insert_f = json.load(open('/Users/koray/Github/fourfront/src/encoded/tests/data/inserts/workflow.json'))
# wf_uuids = [i['uuid'] for i in store['workflow']]
# updated_wf = [i for i in insert_f if i['uuid'] in wf_uuids]
# store['workflow'] = updated_wf

In [ ]:
# if the item exist in the target, should it overwrite it (will include user/award etc)
overwrite_existing = True

# reverse lookup dictionary for schema names
rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]

second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    
    print(len(store[a_type]), 'items exist on source')
    posted = 0
    skip_exist = 0
    for an_item in store[a_type]:
        exists = False
        try:
            # TODO check with all identifiers
            existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
            exists = True
        except:
            exists = False
        # skip the items that exists, if overwrite is not allowed, they them out from patch list
        if exists and existing:
            skip_exist += 1
            if not overwrite_existing:
                remove_existing_items.append(an_item['uuid'])
            # print("{} {} can not post existing item".format(obj_type, an_item['uuid']))
            continue
        posted += 1
        post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
        ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(posted, 'items posted,', skip_exist, 'existing items skipped')
    print(len(second_round_items[a_type]), 'items will be patched in second round')
    print()
    

In [ ]:
# Round II - patch the rest of the metadata
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        print(a_type, '- no items to patch')
        continue 
    for an_item in second_round_items[a_type]:
        if a_type == 'file_fastq':
            if 'extra_files' in an_item:
                del an_item['extra_files']
        ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)
    print(a_type, '- patched')